In [10]:
!pip install pandas sqlalchemy pymysql psycopg2
import pandas as pd
!pip install ace-tools


In [11]:
from sqlalchemy import create_engine


# Database credentials
host = "localhost"
user = "root"
password = "hello"  
database = "fitness"

# Create a database connection
try:
    engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{database}")
    query = "SELECT * FROM exercise"
    df = pd.read_sql(query, engine)
    df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
    # print("successfully")
    print("✅ Successfully connected and retrieved data!")
except Exception as e:
    print("Error:", e)


✅ Successfully connected and retrieved data!


In [12]:
df

,id,name,type,muscle,equipment,difficulty,category,instructions
0,1,smith machine shrug,strength,traps,machine,intermediate,back,"to begin, set the bar height on the smith mach..."
1,2,leverage shrug,strength,traps,machine,intermediate,back,load the pins to an appropriate weight. positi...
2,3,standing dumbbell shrug,strength,traps,dumbbell,intermediate,back,stand erect with a dumbbell on each hand (palm...
3,4,standing dumbbell upright row,strength,traps,dumbbell,intermediate,back,grasp a dumbbell in each hand with a pronated ...
4,5,kettlebell sumo deadlift high pull,strength,traps,kettlebells,intermediate,back,place a kettlebell on the ground between your ...
...,...,...,...,...,...,...,...,...
175,176,bus driver exercise,strength,shoulders,weight_plate,intermediate,shoulders,hold a weight plate in front of you with both ...
176,177,upright rows,strength,shoulders,barbell,intermediate,shoulders,hold a barbell with an overhand grip. pull it ...
177,178,y-raises,strength,shoulders,dumbbell,intermediate,shoulders,"lie face down on an incline bench, holding lig..."
178,179,dumbbell shrugs,strength,shoulders,dumbbell,beginner,shoulders,hold a dumbbell in each hand by your sides. sh...


In [17]:
df['type'].unique()
df['difficulty'].unique()


array(['intermediate', 'beginner', 'expert', 'advanced'], dtype=object)

In [ ]:
df['difficulty'].unique()

array(['intermediate', 'beginner', 'expert', 'advanced'], dtype=object)

In [15]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

user_preferences = {
    "goal": "build muscle",
    "experience": "advanced",
    "equipment": "dumbbell",
    "muscle": "chest"
}

df["combined_features"] = df["type"] + " " + df["difficulty"] + " " + df["equipment"] + " " + df["muscle"] + " " + df["category"]

vectorizer = TfidfVectorizer()
feature_matrix = vectorizer.fit_transform(df["combined_features"])

user_query = f"{user_preferences['goal']} {user_preferences['experience']} {user_preferences['equipment']} {user_preferences['muscle']}"

user_vector = vectorizer.transform([user_query])

similarities = cosine_similarity(user_vector, feature_matrix)

df["similarity_score"] = similarities[0]
recommendations = df.sort_values(by="similarity_score", ascending=False).head(5)

print("Top Workout Recommendations:")
print(recommendations[["name", "difficulty", "equipment", "muscle", "similarity_score"]])


Top Workout Recommendations:
                             name    difficulty equipment muscle  \
104  incline dumbbell bench press  intermediate  dumbbell  chest   
108        decline dumbbell flyes  intermediate  dumbbell  chest   
100          dumbbell bench press  intermediate  dumbbell  chest   
103                dumbbell flyes  intermediate  dumbbell  chest   
102        close-grip bench press  intermediate   barbell  chest   

     similarity_score  
104           0.65723  
108           0.65723  
100           0.65723  
103           0.65723  
102           0.51895  


In [16]:
from IPython.display import display

display(recommendations[["name", "difficulty", "equipment", "muscle", "similarity_score"]])


,name,difficulty,equipment,muscle,similarity_score
104,incline dumbbell bench press,intermediate,dumbbell,chest,0.65723
108,decline dumbbell flyes,intermediate,dumbbell,chest,0.65723
100,dumbbell bench press,intermediate,dumbbell,chest,0.65723
103,dumbbell flyes,intermediate,dumbbell,chest,0.65723
102,close-grip bench press,intermediate,barbell,chest,0.51895


In [4]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

# Database connection function - you already have this working
def load_exercise_data():
    """
    Connect to the database and load exercise data
    """
    from sqlalchemy import create_engine
    
    # Database credentials
    host = "localhost"
    user = "root"
    password = "hello"
    database = "fitness"
    
    try:
        engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{database}")
        query = "SELECT * FROM exercise"
        df = pd.read_sql(query, engine)
        df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
        print("✅ Successfully connected and retrieved data!")
        return df
    except Exception as e:
        print("Error:", e)
        return None

# Function to calculate appropriate workout size based on user details
def calculate_workout_size(user_details, user_preferences):
    """
    Calculate how many exercises should be in the workout plan based on
    user experience, goal, and other factors.
    """
    # Base number of exercises
    base_count = 5
    
    # Adjust based on experience level
    experience_modifier = {
        'beginner': -1,
        'intermediate': 0,
        'advanced': 1
    }
    
    # Adjust based on goal
    goal_modifier = {
        'lose weight': 1,  # More exercises for cardio/weight loss
        'build muscle': 0,
        'strength': -1,    # Fewer but more intense exercises for strength
        'endurance': 2     # More exercises for endurance
    }
    
    # Get modifiers (with defaults if keys don't exist)
    exp_mod = experience_modifier.get(user_preferences.get('experience', 'intermediate'), 0)
    goal_mod = goal_modifier.get(user_preferences.get('goal', 'build muscle'), 0)
    
    # Calculate total exercises
    total_exercises = base_count + exp_mod + goal_mod
    
    # Adjust for number of muscle groups
    muscles = user_preferences.get('muscle', '').split(',')
    num_muscles = len(muscles)
    
    # Ensure at least 1 exercise per muscle group
    return max(total_exercises, num_muscles)

# Main recommendation class
class WorkoutRecommender:
    def __init__(self, df):
        self.df = df
        self.prepare_data()
        self.train_test_split()
        self.build_model()
    
    def prepare_data(self):
        """Prepare data for content-based filtering"""
        # Create combined features for vectorization
        self.df["combined_features"] = (
            self.df["type"].fillna("") + " " + 
            self.df["difficulty"].fillna("") + " " + 
            self.df["equipment"].fillna("") + " " + 
            self.df["muscle"].fillna("") + " " + 
            self.df["category"].fillna("")
        )
    
    def train_test_split(self):
        """Split data into training and testing sets"""
        # Since this is content-based filtering, we'll just use 80% of data for training
        self.train_df, self.test_df = train_test_split(
            self.df, test_size=0.2, random_state=42
        )
    
    def build_model(self):
        """Build the TF-IDF vectorizer model"""
        self.vectorizer = TfidfVectorizer(stop_words='english')
        self.feature_matrix = self.vectorizer.fit_transform(self.train_df["combined_features"])
    
    def get_recommendations(self, user_details, user_preferences, num_recommendations=5):
        """
        Get workout recommendations based on user details and preferences
        
        Args:
            user_details: dict with keys 'age', 'gender', 'height', 'weight'
            user_preferences: dict with keys 'goal', 'experience', 'equipment', 'muscle'
            num_recommendations: number of recommendations to return
            
        Returns:
            DataFrame with recommended exercises
        """
        # Handle multiple muscle groups
        muscles = user_preferences.get('muscle', '').split(',')
        
        # Create a list to store all recommendations
        all_recommendations = []
        
        # Get recommendations for each muscle group
        for muscle in muscles:
            muscle = muscle.strip()
            
            # Adjust user preferences for this specific muscle
            current_preferences = user_preferences.copy()
            current_preferences['muscle'] = muscle
            
            # Create user query for this muscle
            user_query = (
                f"{current_preferences.get('goal', '')} "
                f"{current_preferences.get('experience', '')} "
                f"{current_preferences.get('equipment', '')} "
                f"{muscle}"
            )
            
            # Transform user query into vector space
            user_vector = self.vectorizer.transform([user_query])
            
            # Calculate similarities
            similarities = cosine_similarity(user_vector, self.feature_matrix)
            
            # Add similarity scores to training dataframe (temporary)
            temp_df = self.train_df.copy()
            temp_df["similarity_score"] = similarities[0]
            
            # Filter by specific muscle if provided
            if muscle:
                # Partial matching for muscle to handle cases like "chest" matching "upper chest"
                muscle_filtered = temp_df[temp_df['muscle'].str.contains(muscle, na=False)]
                
                # If no exercises found for this muscle, use original recommendations
                if not muscle_filtered.empty:
                    temp_df = muscle_filtered
            
            # Get top recommendations for this muscle
            muscle_recommendations = temp_df.sort_values(
                by="similarity_score", ascending=False
            ).head(max(3, num_recommendations // len(muscles)))
            
            # Add to our overall recommendations
            all_recommendations.append(muscle_recommendations)
        
        # Combine all recommendations
        if all_recommendations:
            recommendations = pd.concat(all_recommendations)
            
            # Remove duplicates and sort by similarity score
            recommendations = recommendations.drop_duplicates(subset=['name']).sort_values(
                by="similarity_score", ascending=False
            )
            
            return recommendations.head(num_recommendations)
        else:
            return pd.DataFrame()
    
    def create_workout_plan(self, user_details, user_preferences):
        """
        Create a complete workout plan based on user details and preferences
        
        Args:
            user_details: dict with keys 'age', 'gender', 'height', 'weight'
            user_preferences: dict with keys 'goal', 'experience', 'equipment', 'muscle'
            
        Returns:
            DataFrame with recommended workout plan
        """
        # Calculate appropriate workout size
        workout_size = calculate_workout_size(user_details, user_preferences)
        
        # Get recommendations
        recommendations = self.get_recommendations(
            user_details, user_preferences, num_recommendations=workout_size
        )
        
        if recommendations.empty:
            return pd.DataFrame(), "No suitable exercises found"
        
        # Format the plan
        plan = recommendations[["name", "muscle", "equipment", "difficulty", "similarity_score"]]
        
        # Add sets and reps based on goal and experience
        plan = self.add_sets_and_reps(plan, user_preferences)
        
        return plan, "Success"
    
    def add_sets_and_reps(self, plan_df, user_preferences):
        """Add recommended sets and reps based on user goal and experience"""
        goal = user_preferences.get('goal', 'build muscle')
        experience = user_preferences.get('experience', 'intermediate')
        
        # Define sets and reps based on goal
        if goal == 'build muscle':
            sets = 4
            rep_range = "8-12"
        elif goal == 'strength':
            sets = 5
            rep_range = "4-6"
        elif goal == 'endurance':
            sets = 3
            rep_range = "15-20"
        elif goal == 'lose weight':
            sets = 3
            rep_range = "12-15"
        else:
            # Default values
            sets = 3
            rep_range = "10-12"
        
        # Adjust based on experience
        if experience == 'beginner':
            sets = max(2, sets - 1)
        elif experience == 'advanced':
            sets = sets + 1
        
        # Add to dataframe
        plan_df['sets'] = sets
        plan_df['reps'] = rep_range
        
        return plan_df

# Example usage
if __name__ == "__main__":
    # Load data
    exercise_df = load_exercise_data()
    
    if exercise_df is not None:
        # Create recommender
        recommender = WorkoutRecommender(exercise_df)
        
        # Example user details
        user_details = {
            "age": 28,
            "gender": "male",
            "height": 180,  # cm
            "weight": 75    # kg
        }
        
        # Example user preferences with multiple muscle groups
        user_preferences = {
            "goal": "build muscle",
            "experience": "intermediate",
            "equipment": "machine",
            "muscle": "chest,traps"  # Multiple muscle groups
        }
        
        # Get workout plan
        workout_plan, status = recommender.create_workout_plan(user_details, user_preferences)
        
        if status == "Success":
            print("\n=== PERSONALIZED WORKOUT PLAN ===")
            print(f"Goal: {user_preferences['goal']}")
            print(f"Experience: {user_preferences['experience']}")
            print(f"Equipment: {user_preferences['equipment']}")
            print(f"Target muscles: {user_preferences['muscle']}")
            print("\nRECOMMENDED EXERCISES:")
            print(workout_plan[["name", "muscle", "sets", "reps", "similarity_score"]].to_string(index=False))
        else:
            print(status)

✅ Successfully connected and retrieved data!

=== PERSONALIZED WORKOUT PLAN ===
Goal: build muscle
Experience: intermediate
Equipment: machine
Target muscles: chest,traps

RECOMMENDED EXERCISES:
                             name muscle  sets reps  similarity_score
              smith machine shrug  traps     4 8-12          0.969701
                   leverage shrug  traps     4 8-12          0.969701
      calf-machine shoulder shrug  traps     4 8-12          0.861703
                        chest dip  chest     4 8-12          0.740442
barbell bench press - medium grip  chest     4 8-12          0.707727


C:\Users\Ayush\AppData\Local\Temp\ipykernel_11980\3488060140.py:234: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plan_df['sets'] = sets
C:\Users\Ayush\AppData\Local\Temp\ipykernel_11980\3488060140.py:235: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plan_df['reps'] = rep_range


In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

# Database connection function - you already have this working
def load_exercise_data():
    """
    Connect to the database and load exercise data
    """
    from sqlalchemy import create_engine
    
    # Database credentials
    host = "localhost"
    user = "root"
    password = "hello"
    database = "fitness"
    
    try:
        engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{database}")
        query = "SELECT * FROM exercise"
        df = pd.read_sql(query, engine)
        df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
        print("✅ Successfully connected and retrieved data!")
        return df
    except Exception as e:
        print("Error:", e)
        return None

# Function to calculate appropriate workout size based on user details
def calculate_workout_size(user_details, user_preferences):
    """
    Calculate how many exercises should be in the workout plan based on
    user experience, goal, and other factors.
    """
    # Base number of exercises
    base_count = 5
    
    # Adjust based on experience level
    experience_modifier = {
        'beginner': -1,
        'intermediate': 0,
        'advanced': 1
    }
    
    # Adjust based on goal
    goal_modifier = {
        'lose weight': 1,  # More exercises for cardio/weight loss
        'build muscle': 0,
        'strength': -1,    # Fewer but more intense exercises for strength
        'endurance': 2     # More exercises for endurance
    }
    
    # Get modifiers (with defaults if keys don't exist)
    exp_mod = experience_modifier.get(user_preferences.get('experience', 'intermediate'), 0)
    goal_mod = goal_modifier.get(user_preferences.get('goal', 'build muscle'), 0)
    
    # Calculate total exercises
    total_exercises = base_count + exp_mod + goal_mod
    
    # Adjust for number of muscle groups
    muscles = user_preferences.get('muscle', '').split(',')
    num_muscles = len(muscles)
    
    # Ensure at least 1 exercise per muscle group
    return max(total_exercises, num_muscles)

# Main recommendation class
class WorkoutRecommender:
    def __init__(self, df):
        self.df = df
        self.prepare_data()
        self.train_test_split()
        self.build_model()
    
    def prepare_data(self):
        """Prepare data for content-based filtering"""
        # Create combined features for vectorization
        self.df["combined_features"] = (
            self.df["type"].fillna("") + " " + 
            self.df["difficulty"].fillna("") + " " + 
            self.df["equipment"].fillna("") + " " + 
            self.df["muscle"].fillna("") + " " + 
            self.df["category"].fillna("")
        ) 
    
    def train_test_split(self):
        """Split data into training and testing sets"""
        # Since this is content-based filtering, we'll just use 80% of data for training
        self.train_df, self.test_df = train_test_split(
            self.df, test_size=0.2, random_state=42
        )
    
    def build_model(self):
        """Build the TF-IDF vectorizer model"""
        self.vectorizer = TfidfVectorizer(stop_words='english')
        self.feature_matrix = self.vectorizer.fit_transform(self.train_df["combined_features"])
    
    def get_recommendations(self, user_details, user_preferences, num_recommendations=5):
        """
        Get workout recommendations based on user details and preferences
        
        Args:
            user_details: dict with keys 'age', 'gender', 'height', 'weight'
            user_preferences: dict with keys 'goal', 'experience', 'equipment', 'muscle'
            num_recommendations: number of recommendations to return
            
        Returns:
            DataFrame with recommended exercises
        """
        # Handle multiple muscle groups
        muscles = user_preferences.get('muscle', '').split(',')
        
        # Create a list to store all recommendations
        all_recommendations = []
        
        # Get recommendations for each muscle group
        for muscle in muscles:
            muscle = muscle.strip()
            
            # Adjust user preferences for this specific muscle
            current_preferences = user_preferences.copy()
            current_preferences['muscle'] = muscle
            
            # Create user query for this muscle
            user_query = (
                f"{current_preferences.get('goal', '')} "
                f"{current_preferences.get('experience', '')} "
                f"{current_preferences.get('equipment', '')} "
                f"{muscle}"
            )
            
            # Transform user query into vector space
            user_vector = self.vectorizer.transform([user_query])
            
            # Calculate similarities
            similarities = cosine_similarity(user_vector, self.feature_matrix)
            
            # Add similarity scores to training dataframe (temporary)
            temp_df = self.train_df.copy()
            temp_df["similarity_score"] = similarities[0]
            
            # Filter by specific muscle if provided
            if muscle:
                # Partial matching for muscle to handle cases like "chest" matching "upper chest"
                muscle_filtered = temp_df[temp_df['muscle'].str.contains(muscle, na=False)]
                
                # If no exercises found for this muscle, use original recommendations
                if not muscle_filtered.empty:
                    temp_df = muscle_filtered
            
            # Get top recommendations for this muscle
            muscle_recommendations = temp_df.sort_values(
                by="similarity_score", ascending=False
            ).head(max(3, num_recommendations // len(muscles)))
            
            # Add to our overall recommendations
            all_recommendations.append(muscle_recommendations)
        
        # Combine all recommendations
        if all_recommendations:
            recommendations = pd.concat(all_recommendations)
            
            # Remove duplicates and sort by similarity score
            recommendations = recommendations.drop_duplicates(subset=['name']).sort_values(
                by="similarity_score", ascending=False
            )
            
            return recommendations.head(num_recommendations)
        else:
            return pd.DataFrame()
    
    def create_workout_plan(self, user_details, user_preferences):
        """
        Create a complete workout plan based on user details and preferences
        
        Args:
            user_details: dict with keys 'age', 'gender', 'height', 'weight'
            user_preferences: dict with keys 'goal', 'experience', 'equipment', 'muscle'
            
        Returns:
            DataFrame with recommended workout plan
        """
        # Calculate appropriate workout size
        workout_size = calculate_workout_size(user_details, user_preferences)
        
        # Get recommendations
        recommendations = self.get_recommendations(
            user_details, user_preferences, num_recommendations=workout_size
        )
        
        if recommendations.empty:
            return pd.DataFrame(), "No suitable exercises found"
        
        # Format the plan without sets and reps
        plan = recommendations[["name", "muscle", "equipment", "difficulty", "similarity_score"]]
        
        return plan, "Success"

# Example usage
if __name__ == "__main__":
    # Load data
    exercise_df = load_exercise_data()
    
    if exercise_df is not None:
        # Create recommender
        recommender = WorkoutRecommender(exercise_df)
        
        # Example user details
        user_details = {
            "age": 28,
            "gender": "male",
            "height": 180,  # cm
            "weight": 150    # kg
        }
        
        # Example user preferences with multiple muscle groups
        user_preferences = {
            "goal": "lose weight",
            "experience": "advance",
            "equipment": "machine",
            "muscle": "back, biceps"  # Multiple muscle groups
        }
        
        # Get workout plan
        workout_plan, status = recommender.create_workout_plan(user_details, user_preferences)
        
        if status == "Success":
            print("\n=== PERSONALIZED WORKOUT PLAN ===")
            print(f"Goal: {user_preferences['goal']}")
            print(f"Experience: {user_preferences['experience']}")
            print(f"Equipment: {user_preferences['equipment']}")
            print(f"Target muscles: {user_preferences['muscle']}")
            print("\nRECOMMENDED EXERCISES:")
            print(workout_plan[["name", "muscle", "equipment", "difficulty", "similarity_score"]].to_string(index=False))
        else:
            print(status)

✅ Successfully connected and retrieved data!

=== PERSONALIZED WORKOUT PLAN ===
Goal: lose weight
Experience: advance
Equipment: machine
Target muscles: back, biceps

RECOMMENDED EXERCISES:
                               name      muscle equipment   difficulty  similarity_score
                       barbell curl      biceps   barbell intermediate          0.514455
                 ez-bar spider curl      biceps   barbell intermediate          0.514455
             wide-grip barbell curl      biceps   barbell     beginner          0.499352
                              rower middle_back   machine intermediate          0.448061
         reverse-grip bent-over row middle_back   barbell intermediate          0.000000
alternating sit-through with crunch middle_back     other intermediate          0.000000


In [ ]:
# import pandas as pd
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity

# user_preferences = {
#     "goal": "build muscle",
#     "experience": "advanced",
#     "equipment": "dumbbell",
#     "muscle": "chest"
# }

# df["combined_features"] = df["type"] + " " + df["difficulty"] + " " + df["equipment"] + " " + df["muscle"] + " " + df["category"]

# vectorizer = TfidfVectorizer()
# feature_matrix = vectorizer.fit_transform(df["combined_features"])

# user_query = f"{user_preferences['goal']} {user_preferences['experience']} {user_preferences['equipment']} {user_preferences['muscle']}"

# user_vector = vectorizer.transform([user_query])

# similarities = cosine_similarity(user_vector, feature_matrix)

# df["similarity_score"] = similarities[0]
# recommendations = df.sort_values(by="similarity_score", ascending=False).head(5)

# print("Top Workout Recommendations:")
# print(recommendations[["name", "difficulty", "equipment", "muscle", "similarity_score"]])


Top Workout Recommendations:
                             name    difficulty equipment muscle  \
104  incline dumbbell bench press  intermediate  dumbbell  chest   
108        decline dumbbell flyes  intermediate  dumbbell  chest   
100          dumbbell bench press  intermediate  dumbbell  chest   
103                dumbbell flyes  intermediate  dumbbell  chest   
102        close-grip bench press  intermediate   barbell  chest   

     similarity_score  
104           0.65723  
108           0.65723  
100           0.65723  
103           0.65723  
102           0.51895  


In [6]:
# Cell 1: Import libraries
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sqlalchemy import create_engine

host = "localhost"
user = "root"
password = "hello"
database = "fitness"

try:
    engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{database}")
    query = "SELECT * FROM exercise"
    df = pd.read_sql(query, engine)
    df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
    print("✅ Successfully connected and retrieved data!")
except Exception as e:
    print("Error:", e)
    df = None

if df is not None:
    df["combined_features"] = (
        df["type"].fillna("") + " " + 
        df["difficulty"].fillna("") + " " + 
        df["equipment"].fillna("") + " " + 
        df["muscle"].fillna("") + " " + 
        df["category"].fillna("")
    )
    
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
    
    vectorizer = TfidfVectorizer(stop_words='english')
    feature_matrix = vectorizer.fit_transform(train_df["combined_features"])
    
    print("Data preparation complete!")

    import pickle

# Save vectorizer and model
pickle.dump(vectorizer, open('vectorizer.pkl', 'wb'))
df.to_pickle('model.pkl')



✅ Successfully connected and retrieved data!
Data preparation complete!


In [4]:

# Cell 4: Define user details and preferences
# Example user details
user_details = {
    "age": 28,
    "gender": "male",
    "height": 180,  # cm
    "weight": 150   # kg
}

# Example user preferences with multiple muscle groups
user_preferences = {
    "goal": "lose weight",
    "experience": "advance",
    "equipment": "machine",
    "muscle": "back, biceps"  # Multiple muscle groups
}

# Cell 5: Calculate workout size
# Base number of exercises
base_count = 5

# Adjust based on experience level
experience_modifier = {
    'beginner': -1,
    'intermediate': 0,
    'advanced': 1
}

# Adjust based on goal
goal_modifier = {
    'lose weight': 1,  # More exercises for cardio/weight loss
    'build muscle': 0,
    'strength': -1,    # Fewer but more intense exercises for strength
    'endurance': 2     # More exercises for endurance
}

# Get modifiers (with defaults if keys don't exist)
exp_mod = experience_modifier.get(user_preferences.get('experience', 'intermediate'), 0)
goal_mod = goal_modifier.get(user_preferences.get('goal', 'build muscle'), 0)

# Calculate total exercises
total_exercises = base_count + exp_mod + goal_mod

# Adjust for number of muscle groups
muscles = user_preferences.get('muscle', '').split(',')
num_muscles = len(muscles)

# Ensure at least 1 exercise per muscle group
workout_size = max(total_exercises, num_muscles)

print(f"Recommended workout size: {workout_size} exercises")

# Cell 6: Generate recommendations
# Handle multiple muscle groups
muscles = user_preferences.get('muscle', '').split(',')

# Create a list to store all recommendations
all_recommendations = []

# Get recommendations for each muscle group
for muscle in muscles:
    muscle = muscle.strip()
    
    # Adjust user preferences for this specific muscle
    current_preferences = user_preferences.copy()
    current_preferences['muscle'] = muscle
    
    # Create user query for this muscle
    user_query = (
        f"{current_preferences.get('goal', '')} "
        f"{current_preferences.get('experience', '')} "
        f"{current_preferences.get('equipment', '')} "
        f"{muscle}"
    )
    
    # Transform user query into vector space
    user_vector = vectorizer.transform([user_query])
    
    # Calculate similarities
    similarities = cosine_similarity(user_vector, feature_matrix)
    
    # Add similarity scores to training dataframe (temporary)
    temp_df = train_df.copy()
    temp_df["similarity_score"] = similarities[0]
    
    # Filter by specific muscle if provided
    if muscle:
        # Partial matching for muscle to handle cases like "chest" matching "upper chest"
        muscle_filtered = temp_df[temp_df['muscle'].str.contains(muscle, na=False)]
        
        # If no exercises found for this muscle, use original recommendations
        if not muscle_filtered.empty:
            temp_df = muscle_filtered
    
    # Get top recommendations for this muscle
    muscle_recommendations = temp_df.sort_values(
        by="similarity_score", ascending=False
    ).head(max(3, workout_size // len(muscles)))
    
    # Add to our overall recommendations
    all_recommendations.append(muscle_recommendations)

# Cell 7: Combine and display final workout plan
if all_recommendations:
    recommendations = pd.concat(all_recommendations)
    
    # Remove duplicates and sort by similarity score
    recommendations = recommendations.drop_duplicates(subset=['name']).sort_values(
        by="similarity_score", ascending=False
    )
    
    # Get the final workout plan
    workout_plan = recommendations.head(workout_size)
    
    # Format the plan
    final_plan = workout_plan[["name", "muscle", "equipment", "difficulty", "similarity_score"]]
    
    print("\n=== PERSONALIZED WORKOUT PLAN ===")
    print(f"Goal: {user_preferences['goal']}")
    print(f"Experience: {user_preferences['experience']}")
    print(f"Equipment: {user_preferences['equipment']}")
    print(f"Target muscles: {user_preferences['muscle']}")
    print("\nRECOMMENDED EXERCISES:")
    print(final_plan.to_string(index=False))
else:
    print("No suitable exercises found")

Recommended workout size: 6 exercises

=== PERSONALIZED WORKOUT PLAN ===
Goal: lose weight
Experience: advance
Equipment: machine
Target muscles: back, biceps

RECOMMENDED EXERCISES:
                               name      muscle equipment   difficulty  similarity_score
                       barbell curl      biceps   barbell intermediate          0.514455
                 ez-bar spider curl      biceps   barbell intermediate          0.514455
             wide-grip barbell curl      biceps   barbell     beginner          0.499352
                              rower middle_back   machine intermediate          0.448061
         reverse-grip bent-over row middle_back   barbell intermediate          0.000000
alternating sit-through with crunch middle_back     other intermediate          0.000000
